### **Install OpenAI**

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_folder='/content/drive/MyDrive/ll_color_colabs/_projects/_cl200_color_colabs/_eclipse/yellow_eclipse/yellow_eclipse_output'

In [ ]:
import requests

# Assuming this part receives the token successfully as before
from google.colab import userdata
bot_token = userdata.get('SLACK_BOT_TOKEN')

# The channel ID you want to get the messages from
channel_id = userdata.get('slack_channel_id')

# Headers for the API request
headers = {
    'Authorization': f'Bearer {bot_token}',
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Parameters for fetching the last 20 messages
params = {
    'channel': channel_id,
    'limit': 20  # Number of messages to fetch
}

# Fetch the last 20 messages from the specified Slack channel
response = requests.get('https://slack.com/api/conversations.history',
                        headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    messages_data = response.json()  # Parse JSON response
    if messages_data.get("ok"):
        # Iterate and print messages
        for message in messages_data.get("messages", []):
            print(message.get("text"))
    else:
        print(f"Error: {messages_data.get('error')}")
else:
    raise ValueError(
        f'Request to slack returned an error {response.status_code}, the response is:\n{response.text}'
    )

In [4]:
import requests
from openai import OpenAI

def generate_and_slack(system_prompt, user_prompt, channel_id):
    # Assuming OPENAI_API_KEY and SLACK_BOT_TOKEN are fetched securely as shown previously
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
    slack_bot_token = userdata.get('SLACK_BOT_TOKEN')

    # Initialize OpenAI client
    client = OpenAI(api_key=openai_api_key)

    # Generate response using OpenAI
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    generated_text = response.choices[0].message.content
    print(generated_text)

    # Prepare message for Slack
    headers = {
        'Authorization': f'Bearer {slack_bot_token}',
        'Content-Type': 'application/json'
    }

    message = {
        'channel': channel_id,
        'text': generated_text
    }

    # Post the message to the specified Slack channel
    slack_response = requests.post('https://slack.com/api/chat.postMessage',
                              headers=headers, json=message)

    # Verify Slack response
    if slack_response.status_code != 200:
        raise ValueError(
            f'Request to slack returned an error {slack_response.status_code}, the response is:\n{slack_response.text}'
        )
    else:
        print("Message successfully sent to Slack.")



In [ ]:
system_prompt = "you are a village elder"
user_prompt = f"please further this story in a way that will inspire your followers"
channel_id = userdata.get('slack_channel_id')

generate_and_slack(system_prompt, user_prompt, channel_id)